## imports

In [1]:
import os
from geopandas import read_file
from geojson import Feature, FeatureCollection, dump
import json
import simplekml

from pyproj import transform, Proj

data_partition = 'd:\\data\\resowrs'

hexgrid_dir = os.path.join(data_partition, 'hexgrid')
base_file_path = os.path.join(hexgrid_dir, 'grids', 'hex_basegrid.geojson')
out_file_path = os.path.join(hexgrid_dir, 'hexgrid.geojson')

base_grid = read_file(base_file_path)

crs = {
        "type": "name",
        "properties": {"name": base_grid.crs['init']}
      }

def WKT(hexgrid_df):
    
    west = hexgrid_df['west'].values[0]
    east = hexgrid_df['east'].values[0]
    north = hexgrid_df['north'].values[0]
    south = hexgrid_df['south'].values[0]
    
    print(f'POLYGON(({west} {north}, {west} {south} ,' + \
           f'{east} {south}, {east} {north}, {west} {north}))')
    
def CWWKT(hexgrid_df):

    proj_BGS = Proj(init="epsg:27700")
    
    west = hexgrid_df['west'].values[0]
    east = hexgrid_df['east'].values[0]
    north = hexgrid_df['north'].values[0]
    south = hexgrid_df['south'].values[0]
    
    west_4326, north_4326 = proj_BGS(west, north, inverse=True)
    east_4326, south_4326 = proj_BGS(east, south, inverse=True)
    
    print(f'POLYGON(({west_4326} {north_4326}, {west_4326} {south_4326} ,' + \
           f'{east_4326} {south_4326}, {east_4326} {north_4326}, {west_4326} {north_4326}))')

## run the code

In [2]:
features = []
for index, row in base_grid.iterrows():
    features.append(Feature(geometry=row['geometry'],
                            properties={'id': index}))

feature_collection = FeatureCollection(features, crs=crs)

with open(out_file_path, 'w') as f:
   dump(feature_collection, f)

In [3]:
for cell in [324, 325, 389]:
    
    hexcell_file_path = os.path.join(hexgrid_dir, f'hexgrid_{str(cell)}.geojson')

    features = []
    for index, row in base_grid.iterrows():
        
        if index == cell:
            features.append(Feature(geometry=row['geometry'],
                            properties={'id': index,
                                        'west': row['left'],
                                        'east': row['right'],
                                        'south': row['bottom'],
                                        'north': row['top']}))
            print(row['geometry'])
            
    feature_collection = FeatureCollection(features, crs=crs)

    with open(hexcell_file_path, 'w') as f:
       dump(feature_collection, f)
    
    CWWKT(read_file(hexcell_file_path))
    print()

MULTIPOLYGON (((-2.6793451 50.4878638, -2.5968334 50.3983859, -2.4343753 50.3991067, -2.353813 50.4893101, -2.4360247 50.5789604, -2.5991015 50.578235, -2.6793451 50.4878638)))
POLYGON((-2.6793543144087053 50.57721834846131, -2.6793543144087053 50.398786138292195 ,-2.35183102916881 50.398786138292195, -2.35183102916881 50.57721834846131, -2.6793543144087053 50.57721834846131))

MULTIPOLYGON (((-2.5435058 59.3851676, -2.440715 59.295755, -2.237995 59.2962846, -2.1369961 59.38623, -2.2392619 59.4758808, -2.443056 59.475348, -2.5435058 59.3851676)))
POLYGON((-2.543253445098011 59.4754370486306, -2.543253445098011 59.296877423476076 ,-2.1348972173026004 59.296877423476076, -2.1348972173026004 59.4754370486306, -2.543253445098011 59.4754370486306))

MULTIPOLYGON (((-0.4734599 50.7496472, -0.6370873 50.751673, -0.716448 50.8425047, -0.6318393 50.9314746, -0.4675814 50.9294351, -0.3885631 50.8384398, -0.4734599 50.7496472)))
POLYGON((-0.7124779734377259 50.931847486435906, -0.7124779734377259

In [ ]:
with open(base_file_path) as f:
    data = json.load(f)


projectionIn = Proj(init='epsg:32627')

kml = simplekml.Kml()
for feature in data['features']:
    coord_tuples = []
    coord_list = (feature['geometry']['coordinates'][0][0])
    for coord in coord_list:
        long_w, lat_n = projectionIn(coord[0], coord[1], inverse=True)
#        long_w, lat_n = transform(projectionIn, projectionIn,
#                                  coord[0], coord[1])
        coord_tuples.append((long_w, lat_n))
    print(coord_tuples)
    name = str((int(feature['properties']['id'])))
    if feature['geometry']['type'] == 'MultiPolygon':
        kml.newpolygon(name=name,
                       description='test',
                       outerboundaryis=coord_tuples)
kml.save(base_file_path.replace('geojson', 'kml'))

In [ ]:
import rasterio
import pyproj

file = 'S2B_MSIL2A_20210903T144719_N0301_R139_T19LDG_20210903T184347.tif'
# reading the GeoTIFF file and returning the bounds and the coordinate reference
# system (CRS) which is EPS:32719 for Sentinel-2
with rasterio.open(file) as src:
    bounds = src.bounds
    crs_from = src.crs

# defining the CRS to be transformed into
epsg_id = 4326
crs_to = pyproj.CRS.from_epsg(epsg_id)

# defining the transformer which transforms from one CRS to another
proj = pyproj.Transformer.from_crs(crs_from=crs_from, crs_to=crs_to)
bounds_4326 = proj.transform_bounds(*bounds)

print('bounds in EPSG 32719:', bounds)
print('bounds in EPSG 4326:', bounds_4326)